# Chinook Digital Music Shop

In this project, we'll be working on the [Chinook database](https://github.com/lerocha/chinook-database) and using SQL to extract data and answer some business questions.

The database is provided as a SQLite database called *chinook.db*. It contains information about the artists, songs, and albums, as well as information on the shop's employees, customers, and the customers purchases.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Data overview

In [2]:
%%sql
SELECT name, type
  FROM sqlite_master
 WHERE type IN ('table', 'view');

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Below is a schema diagram of the database:

![Database schema diagram](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)

## Selecting Albums to Purchase

The Chinook store has just signed a deal with a new record label, and our task is selecting the first three albums that will be added to the store, from a list of four.

| Artist Name | Genre   |
| ----------- | ------- |
| Regal       | Hip-Hop |
| Red Tone    | Punk    |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

The record label specializes in artists from the USA, so we're interested in finding out which genres sell the best (sell the most tracks) in the USA.

In [3]:
%%sql
WITH usa_sales AS  -- tracks sold in USA
                 (
                  SELECT il.invoice_id, il.track_id
                    FROM invoice_line AS il
                         LEFT JOIN invoice AS i
                         ON il.invoice_id = i.invoice_id
                   WHERE i.billing_country = 'USA'
                 ),
     tracks_sold AS  -- tracks sold in USA by genre
                   (
                    SELECT g.name AS genre,
                           COUNT(u.track_id) AS tracks_sold
                      FROM usa_sales AS u
                           LEFT JOIN track AS t ON u.track_id = t.track_id
                           LEFT JOIN genre AS g ON t.genre_id = g.genre_id
                     GROUP BY g.name
                   )

SELECT *,
       ROUND(CAST(tracks_sold AS Float) 
       / (
          SELECT COUNT(*) FROM usa_sales
         )
       * 100, 2) AS percentage
  FROM tracks_sold
 ORDER BY tracks_sold DESC;

Done.


genre,tracks_sold,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


According to table above, three artists whose albums we should purchase for the store are: **Red Tone**, **Meteor and the Girls**, **Slim Jim Bites**.

## Analyzing Employee Sales Performance

Each customer of the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. Next, we'll analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

Let's find the total amount of sales assigned to each sales support agent, along with their hire date.

In [4]:
%%sql
WITH sales AS    -- total sales per employee
             (
              SELECT SUM(i.total) AS total_sales,
                     c.support_rep_id
                FROM invoice AS i
                     LEFT JOIN customer AS c
                     ON i.customer_id = c.customer_id
               GROUP BY c.support_rep_id
             )

SELECT e.employee_id,
       e.first_name || ' ' || e.last_name AS employee_name,
       s.total_sales,
       e.hire_date
  FROM sales AS s
       LEFT JOIN employee AS e
       ON s.support_rep_id = e.employee_id;

Done.


employee_id,employee_name,total_sales,hire_date
3,Jane Peacock,1731.5100000000039,2017-04-01 00:00:00
4,Margaret Park,1584.0000000000034,2017-05-03 00:00:00
5,Steve Johnson,1393.920000000002,2017-10-17 00:00:00


The result shows some differences in each employee's total sales. However, it can be explained by looking at the hire dates: an employee who worked for a longer period of time has a higher number of total sales assigned. In other words, there is no evidence indicates who had the best or worse performance.

## Sales by Country

The Chinook store wants to extract the sales data to see how customers from different countries are spending. In particular, the task is to calculate data, for each country, on the:
 - Total number of customers
 - Total value of sales
 - Average value of sales per customer
 - Average order value

Countries with only one customer are to be collected into an *"Other"* group and put at the very bottom.

In [5]:
%%sql
WITH country_sales AS  -- calculate summary statistics for each country
                     (
                      SELECT c.country AS country_name,
                             COUNT(DISTINCT i.customer_id) AS number_of_customers,
                             SUM(unit_price) AS total_sales,
                             SUM(unit_price) / COUNT(DISTINCT i.customer_id) AS sales_per_customer,
                             SUM(unit_price) / COUNT(DISTINCT il.invoice_id) AS value_per_order
                        FROM invoice_line AS il
                             LEFT JOIN invoice AS i ON il.invoice_id = i.invoice_id
                             LEFT JOIN customer AS c ON i.customer_id = c.customer_id
                       GROUP BY 1
                     )
    
/* Group countries with one customer into "Other" */
SELECT CASE
           WHEN number_of_customers = 1 THEN 'Other'
           ELSE country_name
       END AS country,
       SUM(number_of_customers) AS number_of_customers,
       SUM(total_sales) AS total_sales,
       AVG(sales_per_customer) AS sales_per_customer,
       AVG(value_per_order) AS value_per_order
  FROM (  -- create a sort column to put the "Other" group at the end
        SELECT cs.*,
               CASE
                   WHEN cs.number_of_customers = 1 THEN 1
                   ELSE 0
               END AS sort
          FROM country_sales AS cs
       )
 GROUP BY country
 ORDER BY sort, total_sales DESC;

Done.


country,number_of_customers,total_sales,sales_per_customer,value_per_order
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
India,2,183.1500000000002,91.5750000000001,8.72142857142858
Other,15,1094.9399999999994,72.99599999999995,7.445071062271058


## Album vs Individual Tracks

The Chinook store wants to save money by considering changing its purchasing strategy: Instead of purchasing a whole album from record companies, it would only purchase the most popular tracks from that album. To understand the effect of this decision on overall revenue, we'll need to find out the percentages of customers' individual tracks purchases, comparing to album purchases.

We're going to categorize each invoice as either an album purchase or not by identifying whether each invoice has all the tracks from an album, then calculate the number/percentage of invoices.

In [23]:
%%sql
WITH invoice_track AS
                     (
                      SELECT i.*, t.album_id
                        FROM 
                            (
                             SELECT invoice_id, track_id
                               FROM invoice_line
                              GROUP BY 1
                            ) AS i
                            LEFT JOIN track AS t
                            ON i.track_id = t.track_id
                     ),
     invoice_categorized AS
                           (
                            SELECT it.invoice_id,
                                   CASE     -- comparing tracks from invoice and tracks from album
                                       WHEN
                                           (
                                            SELECT il.track_id
                                              FROM invoice_line AS il
                                             WHERE il.invoice_id = it.invoice_id
                                            
                                            EXCEPT
                                               
                                            SELECT t.track_id 
                                              FROM track AS t 
                                             WHERE t.album_id = it.album_id 
                                           ) IS NULL
                                       AND
                                           (
                                            SELECT t.track_id 
                                              FROM track AS t 
                                             WHERE t.album_id = it.album_id
                                            
                                            EXCEPT
                                            
                                            SELECT il.track_id
                                              FROM invoice_line AS il 
                                             WHERE il.invoice_id = it.invoice_id
                                           ) IS NULL
                                       THEN 'album'
                                       ELSE 'individuals'
                                   END AS purchase_type
                              FROM invoice_track AS it
                           )
            
SELECT purchase_type,
       COUNT(invoice_id) AS number_of_invoices,
       ROUND(CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice_categorized)* 100, 2) AS percentage
  FROM invoice_categorized
 GROUP BY 1;


Done.


purchase_type,number_of_invoices,percentage
album,114,18.57
individuals,500,81.43


**Conclusion**: There is a considerable amount of album purchases from customers (18.57%), so it's probably not a good idea to stop buying albums from record companies, as this could result in a potential loss of revenue.